In [1]:
import pandas as pd
import os
import time
cwd = os.getcwd()


In [2]:
#set NRCELL output file from parser
nrcell_file = 'C:/Users/wongthot/Downloads/RC16_20221208_orig.xlsx'
netAct_name = 'RC16'

#Dataframe for NRCELL sheet
df1 = pd.read_excel(nrcell_file,sheet_name='NRCELL',skiprows=1)
col_list = df1.columns.str.strip()
df1.columns = col_list
selected_col_list = ['MRBTS','NRBTS','NRCELL','chBw', 'nrarfcn','dlMimoMode','frRefList','rModDNList','Item-tddFrameStructure-ulDlDataSlotRatio','cellName','administrativeState',
 'operationalState','cellDepType']
NRCELL_df = df1[[c for c in df1.columns if c in selected_col_list]]



ValueError: Worksheet named 'NRCELL' not found

In [ ]:
NRCELL_df

In [ ]:
#Dataframe for NRCELL_FDD sheet
df2 = pd.read_excel(nrcell_file,sheet_name='NRCELL_FDD',skiprows=1)
col_list = df2.columns.str.strip()
df2.columns = col_list
df2.iloc[ : , [0,1,2,6,8] ]
NRCELL_FDD_df = df2
NRCELL_FDD_df['DN_MRBTS'] = 'PLMN-PLMN/MRBTS-' + NRCELL_FDD_df['MRBTS'].astype(str) + '/NRBTS-' + NRCELL_FDD_df['NRBTS'].astype(str)
NRCELL_FDD_df['DN_NRCELL'] = NRCELL_FDD_df['DN_MRBTS'] + '/NRCELL-' + NRCELL_FDD_df['NRCELL'].astype(str)

In [ ]:
#Dataframe for RMOD_R sheet
df3 = pd.read_excel(nrcell_file,sheet_name='RMOD_R',skiprows=1)
col_list = df3.columns.str.strip()
df3.columns = col_list
selected_c3_list = ['configDN','productCode','productName']
RMOD_df = df3[[c for c in df3.columns if c in selected_c3_list]]
RMOD_df[['productShortCode','tmp']] = RMOD_df['productCode'].str.split('.',expand=True)
RMOD_df = RMOD_df.drop(columns=['tmp'])

In [ ]:
df4 = pd.read_excel(nrcell_file,sheet_name='ASIRMOD_R',skiprows=1)
col_list = df4.columns.str.strip()
df4.columns = col_list
selected_c4_list = ['configDN','productCode','productName']
ASIRMOD_df = df4[[c for c in df4.columns if c in selected_c4_list]]

In [ ]:
#NRCELL_df[NRCELL_df['rModDNList'].isna()].count()

In [ ]:
df5 = pd.read_excel(nrcell_file,sheet_name='MNL_R',skiprows=1)
col_list = df5.columns.str.strip()
df5.columns = col_list
selected_c5_list = ['MRBTS','activeSWReleaseVersion']
MNL_df = df5[[c for c in df5.columns if c in selected_c5_list]]

In [ ]:
#[col for col in df1.columns if 'state' in col.lower()]

In [ ]:
df6 = pd.read_excel(nrcell_file,sheet_name='BBMOD_R',skiprows=1)
col_list = df6.columns.str.strip()
df6.columns = col_list
selected_c6_list = ['MRBTS','configDN','horizontalPosition','verticalPosition','productCode','productName']
BBMOD_df = df6[[c for c in df6.columns if c in selected_c6_list]]
BBMOD_df[['productShortCode','tmp']] = BBMOD_df['productCode'].str.split('.',expand=True)
BBMOD_df = BBMOD_df.drop(columns=['tmp'])

In [ ]:
df7 = pd.read_excel(nrcell_file,sheet_name='SMOD_R',skiprows=1)
col_list = df7.columns.str.strip()
df7.columns = col_list
selected_c7_list = ['MRBTS','configDN','horizontalPosition','verticalPosition','productCode','productName']
SMOD_df = df7[[c for c in df7.columns if c in selected_c7_list]]
SMOD_df[['productShortCode','tmp']] = SMOD_df['productCode'].str.split('.',expand=True)
SMOD_df = SMOD_df.drop(columns=['tmp'])

In [ ]:
#Dataframe for RMODproduct derived from RMOD_R
RMODproduct_df = RMOD_df[['productCode','productName']].drop_duplicates().copy()
RMODproduct_df[['productShortCode','tmp']] = RMODproduct_df['productCode'].str.split('.',expand=True)
RMODproduct_df = RMODproduct_df.drop(columns=['tmp','productCode'])[::-1]
RMODproduct_df = RMODproduct_df.drop_duplicates().dropna().set_index('productShortCode')


In [ ]:
#Dataframe for BBMOD product derived from BBMOD_R
BBMODproduct_df = BBMOD_df[['productCode','productName']].drop_duplicates().copy()
BBMODproduct_df[['productShortCode','tmp']] = BBMODproduct_df['productCode'].str.split('.',expand=True)
BBMODproduct_df = BBMODproduct_df.drop(columns=['productCode','tmp'])[::-1]
BBMODproduct_df = BBMODproduct_df[BBMODproduct_df['productName'] != 'UNKNOWN']
BBMODproduct_df = BBMODproduct_df.drop_duplicates().dropna().set_index('productShortCode')

In [ ]:
#Dataframe for SMOD product derived from SMOD_R
SMODproduct_df = SMOD_df[['productCode','productName']].drop_duplicates().copy()
SMODproduct_df[['productShortCode','tmp']] = SMODproduct_df['productCode'].str.split('.',expand=True)
SMODproduct_df = SMODproduct_df.drop(columns=['productCode','tmp'])[::-1]
#SMODproduct_df = SMODproduct_df[BBMODproduct_df['productName'] != 'UNKNOWN']
SMODproduct_df = SMODproduct_df.drop_duplicates().dropna().set_index('productShortCode')

In [ ]:
RMOD_df = pd.merge(RMOD_df,RMODproduct_df,on='productShortCode',how='left')
RMOD_df['productName_x'] = RMOD_df['productName_y'].fillna(RMOD_df['productName_x'])
RMOD_df = RMOD_df.drop(columns=['productShortCode','productName_y'])
#RMOD_df.columns = ['configDN','productCode','productName']
RMOD_df.rename(columns={'productName_x':'productName'}, inplace = True)

In [ ]:
BBMOD_df = pd.merge(BBMOD_df,BBMODproduct_df,on='productShortCode',how='left')
BBMOD_df['productName_x'] = BBMOD_df['productName_y'].fillna(BBMOD_df['productName_x'])
BBMOD_df = BBMOD_df.drop(columns=['productShortCode','productName_y'])
#BBMOD_df.columns = ['MRBTS','configDN','horizontalPosition','productCode','productName','verticalPosition']
BBMOD_df.rename(columns={'productName_x':'productName'}, inplace = True)
BBMOD_df['productName'] = BBMOD_df['productName'].str[:4]

In [ ]:
SMOD_df = pd.merge(SMOD_df,SMODproduct_df,on='productShortCode',how='left')
SMOD_df['productName_x'] = SMOD_df['productName_y'].fillna(SMOD_df['productName_x'])
SMOD_df = SMOD_df.drop(columns=['productShortCode','productName_y'])
#SMOD_df.columns = ['MRBTS','configDN','horizontalPosition','productCode','productName','verticalPosition']
SMOD_df.rename(columns={'productName_x':'productName'}, inplace = True)
SMOD_df['productName'] = SMOD_df['productName'].str[:4]

In [ ]:
pivot_BBMOD_df = BBMOD_df.drop(columns=['configDN','horizontalPosition','verticalPosition'])
pivot_BBMOD_df = pivot_BBMOD_df.pivot_table(index='MRBTS', columns = ['productName'], aggfunc = 'count').fillna(0)

#Transform pandas pivot table to regular dataframe with 3 steps
pivot_BBMOD_df.columns = pivot_BBMOD_df.columns.droplevel(0)
pivot_BBMOD_df.columns.name = None
pivot_BBMOD_df = pivot_BBMOD_df.reset_index()

In [ ]:
#if 'dlMimomode' in df.columns:
#    print('yes')
#else:
#    print('no')

In [ ]:
#[col for col in df.columns if 'tdd' in col]

In [ ]:
pivot_SMOD_df = SMOD_df.drop(columns=['configDN','horizontalPosition','verticalPosition'])
pivot_SMOD_df = pivot_SMOD_df.pivot_table(index='MRBTS', columns = ['productName'], aggfunc = 'count').fillna(0)

#Transform pandas pivot table to regular dataframe with 2 steps
pivot_SMOD_df.columns = pivot_SMOD_df.columns.droplevel(0)
pivot_SMOD_df = pivot_SMOD_df.reset_index().rename_axis(None, axis=1)

In [ ]:
NRCELL_df['DN_MRBTS'] = 'PLMN-PLMN/MRBTS-' + NRCELL_df['MRBTS'].astype(str) + '/NRBTS-' + NRCELL_df['NRBTS'].astype(str)
NRCELL_df['DN_NRCELL'] = NRCELL_df['DN_MRBTS'] + '/NRCELL-' + NRCELL_df['NRCELL'].astype(str)


In [ ]:
#Generate SiteName, Tech, SectorName for Smart Capex
NRCELL_df[['cellName','cell']] = NRCELL_df['cellName'].str.split(':',expand=True)
NRCELL_df[['SiteName','Tech','Sec','Sector']] = NRCELL_df['cell'].str.split('-',expand=True)
NRCELL_df['SectorName'] = NRCELL_df['SiteName'] + '_' + NRCELL_df['Sector']
NRCELL_df = NRCELL_df.drop(columns=['cellName','cell','Sec','Sector'])

In [ ]:
#Combine frRefList and rModDNList
NRCELL_df['DNList'] = NRCELL_df['frRefList'].fillna(NRCELL_df['rModDNList'])
NRCELL_df[['Temp','configDN','RMOD2']] = NRCELL_df['DNList'].str.split(';',expand=True)
NRCELL_df = NRCELL_df.drop(columns=['frRefList','rModDNList','DNList','Temp'])

In [ ]:
#NRCELL_df = NRCELL_df.iloc[: , :-1]

In [ ]:
#NRCELL_df[NRCELL_df['configDN'].notnull()]

In [ ]:
#Merged Dataframe
merged_df = pd.merge(NRCELL_df, NRCELL_FDD_df[['DN_NRCELL','chBwDl','nrarfcnDl']],on='DN_NRCELL', how='left')
merged_df['chBw']= merged_df['chBwDl'].fillna(merged_df['chBw'])
merged_df['nrarfcn']= merged_df['nrarfcnDl'].fillna(merged_df['nrarfcn'])
merged_df = merged_df.drop(columns=['chBwDl','nrarfcnDl'])
merged_df = pd.merge(merged_df,RMOD_df, on='configDN', how='left')
merged_df = pd.merge(merged_df,ASIRMOD_df, on='configDN', how='left')
merged_df['productName_x']= merged_df['productName_y'].fillna(merged_df['productName_x'])
merged_df = merged_df.drop(columns=['productCode_y','productName_y'])
merged_df = merged_df.rename(columns={'productCode_x': 'productCode', 'productName_x': 'productName'})
merged_df = pd.merge(merged_df,MNL_df, on='MRBTS', how = 'left')
merged_df = pd.merge(merged_df, pivot_SMOD_df, on = 'MRBTS', how = 'left')
merged_df = pd.merge(merged_df, pivot_BBMOD_df, on = 'MRBTS', how = 'left')

In [ ]:
merged_df.head(10)

In [ ]:
sheet_list = ['NRCEL','pv_SMOD','pv_BBMOD']
df_list =[merged_df,pivot_SMOD_df,pivot_BBMOD_df]
TodaysDate = time.strftime("%Y%m%d")
excelfilename = netAct_name + '_NRCEL_Info_' + TodaysDate +".xlsx"


# function to combine multiple dataframes to one excel file multiple sheets

def dfs_to_sheets(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()
    
dfs_to_sheets(df_list, sheet_list, excelfilename)